In [1]:
!pip install -qU typing langchain-community langgraph "langchain-nvidia-ai-endpoints"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
key = userdata.get('llm_projects')

In [3]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from typing import List, Union, TypedDict, Annotated, Sequence
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, ToolMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool

llm  = ChatNVIDIA(
    model ="meta/llama3-70b-instruct" ,
    api_key = key,
    temperature = 0.7,
    top_p = 1,
    max_tokens = 1024)

Agent I : Agent ChatBot with memory

In [4]:
class AgentState(TypedDict):
    messages : List[Union[HumanMessage,AIMessage]]

def process(state:AgentState) -> AgentState:
  response = llm.invoke(state["messages"])
  state["messages"].append(AIMessage(content=response.content))
  print("Current state", state["messages"])
  return state

graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

conversation_history = []

while (user_input := input("Human: ")) != "exit":
  conversation_history.append(HumanMessage(content=user_input))
  agent_response = agent.invoke({"messages": conversation_history})
  conversation_history = agent_response["messages"]
  user_input = input("Human: ")

with open("logging.txt", "w") as file:
    file.write("Your Conversation Log:\n")

    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")
    file.write("End of Conversation")

print("Conversation saved to logging.txt")

Human: exit
Conversation saved to logging.txt


Agent II: ReAct Agent [Reason and Action Agent]

In [7]:
class AgentState(TypedDict):
  message : Annotated[Sequence[BaseMessage],add_messages]



@tool
def add(a:int,b:int):
  """this is an addition function that adds two numbers"""
  return a + b

@tool
def substract(a:int,b:int):
  """this is an subtraction function that subtracts two numbers"""
  return a - b

@tool
def multiply(a:int,b:int):
  """this is an multiplication function that multiplies two numbers"""
  return a * b

tools = [add,substract,multiply]

llm = ChatNVIDIA(
    model ="meta/llama3-70b-instruct" ,
    api_key = key,
    temperature = 0.7,
    top_p = 1,
    max_tokens = 1024).bind_tools(tools)

class AgentState(TypedDict):
  message : Annotated[Sequence[BaseMessage],add_messages]

def model_call(state: AgentState) -> AgentState:
  system_prompt = SystemMessage(
      content="You are an AI asistent, please answer my query to the best of your ability.")
  messages_to_send = [system_prompt] + state["message"]
  response = llm.invoke(messages_to_send)
  return {"message": [response]}

def should_continue(state: AgentState) -> AgentState:
  last_message = state["message"][-1]
  if not last_message.tool_calls:
    return "end"
  else:
    return "continue"

graph = StateGraph(AgentState)
graph.add_node("model_call", model_call)

tool_node = ToolNode(tools=tools)
graph.add_node("tool_node", tool_node)

graph.add_edge(START, "model_call")

graph.add_conditional_edges(
    "model_call",
    should_continue,
    {
        "continue": "tool_node",
        "end": END,
    }
)

graph.add_edge("tool_node", "model_call")

agent = graph.compile()


/usr/local/lib/python3.11/dist-packages/langchain_nvidia_ai_endpoints/chat_models.py:592: UserWarning: Model 'meta/llama3-70b-instruct' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(


In [8]:
def print_stream(stream):
    for s in stream:
        message = s["message"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.")]}
print_stream(agent.stream(inputs, stream_mode="values"))

================================== Ai Message ==================================

I'll do my best to assist you. Please go ahead and ask your question or share your query, and I'll respond with a helpful and accurate answer. I'm here to help!


Agent III : Drafter